# 🎤 TTS Сервер с Edge-TTS и Cloudflare Tunnel

Этот скрипт запускает Text-to-Speech сервер с автоматическим созданием публичного URL через Cloudflare Tunnel.

## Особенности:
- Использует Microsoft Edge TTS (бесплатный, высококачественный)
- Автоматически создает публичный URL
- Поддерживает CORS для веб-сайтов
- 7 различных английских голосов
- Работает в Google Colab и аналогичных средах

## 📦 Установка зависимостей

In [ ]:
print("=" * 80)
print("📦 УСТАНОВКА ЗАВИСИМОСТЕЙ")
print("=" * 80)

print("Устанавливаем edge-tts, Flask и requests...")
!pip install edge-tts flask requests -q

print("✅ Зависимости установлены")

## 🚀 Запуск TTS сервера

In [ ]:
import os
import subprocess
import asyncio
import threading
import time
import re
import sys
import queue
from flask import Flask, request, Response
import edge_tts

print("✅ Модули импортированы")

# Создаем Flask приложение
app = Flask(__name__)

# Включаем CORS
@app.after_request
def cors(response):
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Access-Control-Allow-Methods"] = "GET, OPTIONS"
    response.headers["Access-Control-Allow-Headers"] = "Content-Type"
    return response

@app.route("/")
def tts_root():
    return tts_request()

@app.route("/tts")
def tts_request():
    """Основная функция TTS"""
    text = request.args.get("text", "")
    voice = request.args.get("voice", "us_young")
    
    if not text:
        return "No text provided", 400
    
    # Словарь голосов
    voices = {
        "us_young": "en-US-ChristopherNeural",
        "us_male": "en-US-EricNeural", 
        "us_female": "en-US-JennyNeural",
        "uk_male": "en-GB-RyanNeural",
        "uk_female": "en-GB-SoniaNeural",
        "australian": "en-AU-WilliamNeural",
        "indian": "en-IN-PrabhatNeural"
    }
    
    voice_id = voices.get(voice, voices["us_young"])
    
    async def generate_audio():
        """Генерация аудио"""
        communicate = edge_tts.Communicate(text, voice_id)
        chunks = []
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                chunks.append(chunk["data"])
        return b"".join(chunks)
    
    try:
        audio = asyncio.run(generate_audio())
        return Response(audio, mimetype="audio/mpeg")
    except Exception as e:
        return str(e), 500

# Функция запуска сервера
def run_server():
    app.run(host="0.0.0.0", port=9999, debug=False, threaded=True, use_reloader=False)

# Убиваем старые процессы на порту 9999
print("Останавливаем старые процессы...")
!fuser -k 9999/tcp 2>/dev/null || true

# Запускаем сервер в отдельном потоке
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

print("✅ TTS сервер запущен на http://localhost:9999")

## 🌐 Установка и запуск Cloudflare Tunnel

In [ ]:
print("=" * 80)
print("🌐 ЗАПУСК CLOUDFLARED")
print("=" * 80)

# Останавливаем старые процессы cloudflared
!pkill -f cloudflared 2>/dev/null || true
time.sleep(2)

# Устанавливаем cloudflared
print("Устанавливаем cloudflared...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb 2>/dev/null || apt-get install -f -y

# Глобальные переменные для хранения URL
cloudflared_output = queue.Queue()
cloudflared_url = None
tunnel_process = None

def run_cloudflared():
    """Запускаем cloudflared и захватываем его вывод"""
    global cloudflared_url, tunnel_process
    
    print("Запускаем Cloudflare Tunnel...")
    
    # Запускаем процесс
    tunnel_process = subprocess.Popen(
        ['cloudflared', 'tunnel', '--url', 'http://localhost:9999'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    
    # Читаем вывод в реальном времени
    while True:
        line = tunnel_process.stdout.readline()
        if not line and tunnel_process.poll() is not None:
            break
        if line:
            cloudflared_output.put(line)
            
            # Выводим важные сообщения
            if 'error' in line.lower() or 'https://' in line.lower() or 'tunnel' in line.lower():
                print(f"[cloudflared] {line.strip()}")
            
            # Ищем URL в выводе
            url_patterns = [
                r'https://[a-zA-Z0-9.-]+\.trycloudflare\.com',
                r'Visit it at.*?(https://[^\s]+)',
                r'Your quick Tunnel.*?(https://[^\s]+)'
            ]
            
            for pattern in url_patterns:
                matches = re.findall(pattern, line, re.IGNORECASE)
                if matches:
                    cloudflared_url = matches[-1]
                    print(f"✅ URL найден: {cloudflared_url}")
                    break

# Запускаем cloudflared в отдельном потоке
tunnel_thread = threading.Thread(target=run_cloudflared, daemon=True)
tunnel_thread.start()

print("\n⏳ Ждем 20 секунд для получения URL...")
time.sleep(20)

# Проверяем, нашли ли URL
if cloudflared_url:
    public_url = cloudflared_url
    print(f"\n🎉 ВАШ РАБОЧИЙ URL: {public_url}")
else:
    # Если не нашли в реальном времени, проверяем вывод вручную
    print("\n🔍 Проверяем вывод cloudflared вручную...")
    all_output = []
    while not cloudflared_output.empty():
        try:
            line = cloudflared_output.get_nowait()
            all_output.append(line)
        except:
            break
    
    full_output = "".join(all_output)
    
    # Ищем URL в полном выводе
    for pattern in [r'https://[a-zA-Z0-9.-]+\.trycloudflare\.com']:
        matches = re.findall(pattern, full_output, re.IGNORECASE)
        if matches:
            public_url = matches[-1]
            print(f"✅ Найден URL: {public_url}")
            break
    
    if 'public_url' not in locals():
        # Если URL не найден, показываем вывод для ручного поиска
        print("\n📋 Последние строки вывода cloudflared:")
        for line in all_output[-10:]:
            print(line.strip())
        
        # Используем примерный URL
        public_url = "https://example.trycloudflare.com"
        print(f"\n⚠️ URL не найден автоматически. Используйте строку с 'https://' из вывода выше.")

print("✅ Cloudflare Tunnel запущен")

## 🧪 Тестирование и информация

In [ ]:
if 'public_url' in locals() and public_url != "https://example.trycloudflare.com":
    print("=" * 80)
    print("🧪 ТЕСТИРОВАНИЕ И ИНФОРМАЦИЯ")
    print("=" * 80)
    
    print(f"\n🎯 ВАШ ПУБЛИЧНЫЙ URL: {public_url}")
    
    # Сохраняем URL в файл
    with open('/content/tts_server_url.txt', 'w') as f:
        f.write(public_url)
    print(f"💾 URL сохранен в: /content/tts_server_url.txt")
    
    # Тестируем сервер
    import requests
    
    print("\n🧪 Тестируем соединение...")
    try:
        test_url = f"{public_url}/tts?text=Hello+World&voice=us_young"
        response = requests.get(test_url, timeout=30)
        
        if response.status_code == 200:
            print(f"✅ Сервер работает! Получено {len(response.content)} байт аудио")
            
            # Сохраняем тестовое аудио
            with open('/content/test_tts.mp3', 'wb') as f:
                f.write(response.content)
            print(f"💾 Тестовое аудио сохранено: /content/test_tts.mp3")
        else:
            print(f"⚠️ Статус ответа: {response.status_code}")
            
    except requests.exceptions.Timeout:
        print("❌ Таймаут при тестировании")
    except requests.exceptions.ConnectionError:
        print("❌ Ошибка подключения")
    except Exception as e:
        print(f"❌ Ошибка: {e}")
    
    # Информация о голосах
    print("\n📋 ДОСТУПНЫЕ ГОЛОСА:")
    voices_info = [
        ("us_young", "Английский (молодой мужской)"),
        ("us_male", "Английский (мужской)"),
        ("us_female", "Английский (женский)"),
        ("uk_male", "Британский (мужской)"),
        ("uk_female", "Британский (женский)"),
        ("australian", "Австралийский"),
        ("indian", "Индийский")
    ]
    
    for voice_id, description in voices_info:
        print(f"• {voice_id:12} - {description}")
    
    # Примеры использования
    print("\n🎯 ПРИМЕРЫ ИСПОЛЬЗОВАНИЯ:")
    print(f"1. TTS эндпоинт: {public_url}/tts?text=Hello+World&voice=us_young")
    print(f"2. Корневой эндпоинт: {public_url}/?text=Hello+World&voice=us_female")
    print(f"3. Длинный текст: {public_url}/tts?text=This+is+a+long+text+example&voice=uk_male")
    
    print("\n💡 ДЛЯ ВЕБ-САЙТА:")
    print(f"Используйте этот URL на вашем сайте: {public_url}")
    
else:
    print("\n⚠️ Не удалось получить рабочий URL.")
    print("Проверьте вывод cloudflared выше и найдите строку с 'https://'")

print("\n" + "=" * 80)
print("✅ НАСТРОЙКА ЗАВЕРШЕНА")
print("=" * 80)

## 🔧 Проверка статуса

In [ ]:
print("=" * 80)
print("🔧 ПРОВЕРКА СТАТУСА")
print("=" * 80)

# Проверяем процессы
print("\n📊 Активные процессы:")
!ps aux | grep -E "(flask|cloudflared|python.*9999)" | grep -v grep

# Проверяем сетевые соединения
print("\n🌐 Сетевые соединения на порту 9999:")
!netstat -tlnp 2>/dev/null | grep :9999 || echo "Порт 9999 не слушает"

# Если есть URL, показываем его
if 'public_url' in locals() and os.path.exists('/content/tts_server_url.txt'):
    with open('/content/tts_server_url.txt', 'r') as f:
        saved_url = f.read().strip()
    print(f"\n💾 Сохраненный URL: {saved_url}")
    
    # Быстрая проверка
    try:
        import requests
        response = requests.get(f"{saved_url}/", timeout=5)
        print(f"✅ Сервер отвечает (статус: {response.status_code})")
    except:
        print("⚠️ Не удалось проверить сервер")

print("\n💡 СЕРВЕР БУДЕТ РАБОТАТЬ ПОКА ЗАПУЩЕН ЭТОТ NOTEBOOK")

## 🎯 Быстрое использование на сайте

Для использования на вашем сайте:

```javascript
// Пример использования на сайте
async function playTTS(text, voice = 'us_young') {
    const url = `ВАШ_URL/tts?text=${encodeURIComponent(text)}&voice=${voice}`;
    
    try {
        const response = await fetch(url);
        if (response.ok) {
            const audioBlob = await response.blob();
            const audioUrl = URL.createObjectURL(audioBlob);
            const audio = new Audio(audioUrl);
            audio.play();
        }
    } catch (error) {
        console.error('Ошибка TTS:', error);
    }
}

// Использование
playTTS('Hello World', 'us_young');
```

Замените `ВАШ_URL` на полученный выше URL.